In [1]:
from odps import ODPS
from odps import options
import oss2
from odps.df import DataFrame
import datetime
import pandas as pd
import numpy as np
import threading
import queue
import time

# connect信息
from ACCESS import ACCESS_ID
from ACCESS import SECRET_ACCESS_KEY
from ACCESS import ODPS_PROJECT
from ACCESS import ODPS_ENDPOINT

# # 此为外网地址
OSS_ENDPOINT = 'http://oss-cn-shanghai.aliyuncs.com'
OSS_BUCKET = 'hypertrons'

# oss实例
auth = oss2.Auth(ACCESS_ID, SECRET_ACCESS_KEY)
bucket = oss2.Bucket(auth, OSS_ENDPOINT, OSS_BUCKET)

# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY, project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

hints = {'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'}

In [20]:
drop_sql = '''
DROP TABLE IF EXISTS node;
CREATE TABLE IF NOT EXISTS node
(
    org_id STRING
);
'''
o.execute_sql(drop_sql, hints=hints)

select_sql = '''
    INSERT INTO node
    SELECT DISTINCT org_id
    FROM Yue_org_degree
    WHERE degree > 850 or (degree > 100 and degree < 200)
    '''
o.execute_sql(select_sql, hints=hints)
table_node = DataFrame(o.get_table('node'))
df_node = table_node.to_pandas()
df_node.rename(columns={'org_id':'org_id:ID'},inplace=True)
df_node.to_csv('data/node.csv',index=False)

In [21]:
drop_sql = '''
DROP TABLE IF EXISTS edge;
CREATE TABLE IF NOT EXISTS edge
(
    actor_org_id STRING,
    repo_org_id STRING,
    type STRING DEFAULT 'PR'
);
'''
o.execute_sql(drop_sql, hints=hints)

select_sql = '''
    INSERT INTO edge (actor_org_id, repo_org_id)
    SELECT DISTINCT actor_org_id, repo_org_id
    FROM Yue_ORG_To_ORG
    WHERE actor_org_id!='0' AND repo_org_id!='0' AND actor_org_id!=repo_org_id 
        AND actor_org_id IN (select org_id from node) 
        AND repo_org_id IN (select org_id from node)
    '''
o.execute_sql(select_sql, hints=hints)
table_edge = DataFrame(o.get_table('edge'))
df_edge = table_edge.to_pandas()

df_edge.rename(columns={'actor_org_id':'actor_org_id:START_ID', 'repo_org_id':'repo_org_id:END_ID', 'type':':TYPE'},inplace=True)
df_edge.to_csv('data/relation.csv',index=False)

#### 头部organization

In [4]:
drop_sql = '''
DROP TABLE IF EXISTS node;
CREATE TABLE IF NOT EXISTS node
(
    org_id STRING
);
'''
o.execute_sql(drop_sql, hints=hints)

select_sql = '''
    INSERT INTO node
    SELECT DISTINCT org_id
    FROM Yue_org_degree
    WHERE degree > 900
    '''
o.execute_sql(select_sql, hints=hints)
table_node = DataFrame(o.get_table('node'))
df_node = table_node.to_pandas()
df_node.rename(columns={'org_id':'org_id:ID'},inplace=True)
df_node.to_csv('data/nodeTop.csv',index=False)

In [5]:
drop_sql = '''
DROP TABLE IF EXISTS edge;
CREATE TABLE IF NOT EXISTS edge
(
    actor_org_id STRING,
    repo_org_id STRING,
    type STRING DEFAULT 'PR'
);
'''
o.execute_sql(drop_sql, hints=hints)

select_sql = '''
    INSERT INTO edge (actor_org_id, repo_org_id)
    SELECT DISTINCT actor_org_id, repo_org_id
    FROM Yue_ORG_To_ORG
    WHERE actor_org_id!='0' AND repo_org_id!='0' AND actor_org_id!=repo_org_id 
        AND actor_org_id IN (select org_id from node) 
        AND repo_org_id IN (select org_id from node)
    '''
o.execute_sql(select_sql, hints=hints)
table_edge = DataFrame(o.get_table('edge'))
df_edge = table_edge.to_pandas()

df_edge.rename(columns={'actor_org_id':'actor_org_id:START_ID', 'repo_org_id':'repo_org_id:END_ID', 'type':':TYPE'},inplace=True)
df_edge.to_csv('data/relationTop.csv',index=False)